In [1]:
import os
import math
import datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import bert
from bert import BertModelLayer
from bert.loader import StockBertConfig, map_stock_config_to_params, load_stock_weights
from bert.tokenization.bert_tokenization import FullTokenizer
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')
def normalize_text(text):
    tm1 = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
    tm2 = re.sub('<code>.*?</code>', '', tm1, flags=re.DOTALL)
    tm3 = re.sub('<[^>]+>©', '', tm1, flags=re.DOTALL)
    return tm3.replace("\n", "")
print('ok')

ok


In [3]:
# Stop words
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
len(spacy_stopwords)
for stop_word in list(spacy_stopwords)[:10]:
    print(stop_word)

when
whereby
often
‘ve
become
‘s
doing
would
that
was


In [4]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
punctuations = string.punctuation
stopwords = STOP_WORDS

In [5]:
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

In [7]:
bert_model_name="uncased_L-12_H-768_A-12"
bert_ckpt_dir = os.path.join("model/", bert_model_name)
bert_ckpt_file = os.path.join(bert_ckpt_dir, "bert_model.ckpt")
bert_config_file = os.path.join(bert_ckpt_dir, "bert_config.json")

In [8]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [9]:
def create_model(max_seq_len, bert_ckpt_file):
    with tf.io.gfile.GFile(bert_config_file, "r") as reader:
        bc = StockBertConfig.from_json_string(reader.read())
        bert_params = map_stock_config_to_params(bc)
        bert_params.adapter_size = None
        bert = BertModelLayer.from_params(bert_params, name="bert")
    input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
    bert_output = bert(input_ids)
    print("bert shape", bert_output.shape)
    cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
    cls_out = keras.layers.Dropout(0.5)(cls_out)
    logits = keras.layers.Dense(units=768, activation="swish")(cls_out)
    logits = keras.layers.Dropout(0.5)(logits)
    logits = keras.layers.Dense(units=8, activation="softmax")(logits)
    model = keras.Model(inputs=input_ids, outputs=logits)
    model.build(input_shape=(None, max_seq_len))
    load_stock_weights(bert, bert_ckpt_file)
    return model

In [10]:
model = create_model(128, bert_ckpt_file)

bert shape (None, 128, 768)
Done loading 196 BERT weights from: model/uncased_L-12_H-768_A-12\bert_model.ckpt into <bert.model.BertModelLayer object at 0x000002330B062C40> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [11]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")]
)

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 128)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 128, 768)          108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                

In [12]:
checkpoint_save_path = "./checkpoint/wine.ckpt"

if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss')

-------------load the model-----------------


In [13]:
train_df = pd.read_csv('./train_df.csv')
test_df = pd.read_csv('./test_df.csv')
classes = train_df.intent.unique().tolist()
print(classes)

['Cabernet Sauvignon', 'Syrah', 'Riesling', 'Chardonnay', 'Pinot Noir', 'Sauvignon Blanc', 'Bordeaux-style Red Blend', 'Red Blend']


In [14]:
print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2020-12-11 03:05:35


In [31]:
sentences = [
  "This is taut and mineral, with hidden power. It's very closed at this stage. It's a wine with acidity and notes of green fruits and crisp apples, with just a touch of wood. This will age for many years",
  "This fruit is delicious. The fruits of the known fruit, darkness and water are filled. That's all. Rich, open, soft and strong ",
  "The aroma weaves a brocade of peaches, butter, grass, peas and citrus, and the taste buds are closely related to texture. With balanced acidity and chalkiness, supporting the taste of lemon and green herbs, this is a complex and easy to eat product"
]
pred_tokens = map(tokenizer.tokenize, sentences)
pred_tokens = map(lambda tok: ["[CLS]"] + tok + ["[SEP]"], pred_tokens)
pred_token_ids = list(map(tokenizer.convert_tokens_to_ids, pred_tokens))
pred_token_ids = map(lambda tids: tids +[0]*(128-len(tids)),pred_token_ids)
pred_token_ids = np.array(list(pred_token_ids))
predictions = model.predict(pred_token_ids).argmax(axis=-1)
for text, label in zip(sentences, predictions):
    print("text:", text, "\nintent:", classes[label])
    print()

text: This is taut and mineral, with hidden power. It's very closed at this stage. It's a wine with acidity and notes of green fruits and crisp apples, with just a touch of wood. This will age for many years 
intent: Chardonnay

text: This fruit is delicious. The fruits of the known fruit, darkness and water are filled. That's all. Rich, open, soft and strong  
intent: Bordeaux-style Red Blend

text: The aroma weaves a brocade of peaches, butter, grass, peas and citrus, and the taste buds are closely related to texture. With balanced acidity and chalkiness, supporting the taste of lemon and green herbs, this is a complex and easy to eat product 
intent: Chardonnay



In [16]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir, "vocab.txt"))

In [17]:
class IntentDetectionData:
    DATA_COLUMN = "text"
    LABEL_COLUMN = "intent"
    def __init__(self, train, test, tokenizer: FullTokenizer, classes, max_seq_len=192):
        self.tokenizer = tokenizer
        self.max_seq_len = 0
        self.classes = classes
        train, test = map(lambda df: df.reindex(df[IntentDetectionData.DATA_COLUMN].str.len().sort_values().index), [train, test])
        ((self.train_x, self.train_y), (self.test_x, self.test_y)) = map(self._prepare, [train, test])
        print("max seq_len", self.max_seq_len)
        self.max_seq_len = min(self.max_seq_len, max_seq_len)
        self.train_x, self.test_x = map(self._pad, [self.train_x, self.test_x])
        
    def _prepare(self, df):
        x, y = [], []
        for _, row in tqdm(df.iterrows()):
            text, label = row[IntentDetectionData.DATA_COLUMN], row[IntentDetectionData.LABEL_COLUMN]
            tokens = self.tokenizer.tokenize(text)
            tokens = ["[CLS]"] + tokens + ["[SEP]"]
            token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            self.max_seq_len = max(self.max_seq_len, len(token_ids))
            x.append(token_ids)
            y.append(self.classes.index(label))
        return np.array(x), np.array(y)
    
    def _pad(self, ids):
        x = []
        for input_ids in ids:
            input_ids = input_ids[:min(len(input_ids), self.max_seq_len - 2)]
            input_ids = input_ids + [0] * (self.max_seq_len - len(input_ids))
            x.append(np.array(input_ids))
        return np.array(x)

In [18]:
data = IntentDetectionData(train_df, test_df, tokenizer, classes, max_seq_len=128)

24368it [00:14, 1695.36it/s]
11976it [00:07, 1698.90it/s]


max seq_len 180


In [19]:
pd.DataFrame(data.train_x,data.train_y).to_csv('train_xy_vec.csv')

In [20]:
test_xy_vec = pd.read_csv('test_xy_vec.csv')

In [21]:
mytest_y = test_xy_vec['128'].values

In [22]:
mytest_x = test_xy_vec[[str(i) for i in range(128)]].values

In [23]:
my_pred = model.predict(mytest_x).argmax(axis=-1)
print(classification_report(mytest_y, my_pred, target_names=classes))

                          precision    recall  f1-score   support

      Cabernet Sauvignon       0.73      0.80      0.76      1747
                   Syrah       0.67      0.67      0.67       782
                Riesling       0.92      0.88      0.90       927
              Chardonnay       0.90      0.86      0.88      2190
              Pinot Noir       0.85      0.81      0.83      2490
         Sauvignon Blanc       0.72      0.85      0.78       949
Bordeaux-style Red Blend       0.79      0.83      0.81      1279
               Red Blend       0.85      0.77      0.81      1612

                accuracy                           0.81     11976
               macro avg       0.80      0.81      0.81     11976
            weighted avg       0.82      0.81      0.82     11976



In [24]:
print('end of load')

end of load
